<h1><center>Spatial Smoothing Process</center></h1> 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
from datetime import timedelta
from tqdm import tqdm

Project_Path='Project/Path'

### Input data

In [ ]:
Distance_Data=pd.read_csv(Project_Path+ '/Data/Distance.csv',sep=',',index_col=0)
Final_Weather_Data=pd.read_csv(Project_Path+ '/Data/Final_Weather_Data.csv',low_memory=False,sep=',',index_col=0)
Legal_illegal=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.3.csv',sep=',',index_col=0)
train_data=pd.read_csv(Project_Path+ '/Data/Train_TimeSeries.csv',sep=',',index_col=0)
test_data=pd.read_csv(Project_Path+ '/Data/Test_TimeSeries.csv',sep=',',index_col=0)
Names=['Slot_id','Slot_Timeint','Ilegality_Rate']

## Temporal Smoothing

The following procedure apply the spatial smoothing process. Essentially using one data sample creates three, that is, it triples the data set. We have described the process in the paper.

###  Calculation of the mean parking violation rate for the given sector and timeslot.

In [ ]:
def Return_Mean(Time_Slot,Sector,train_data,Names):
    Help=train_data[Names]
    ILegal_Sum = Help.groupby(['Slot_id','Slot_Timeint']).Ilegality_Rate.mean().reset_index()
    ILegal_Sum=ILegal_Sum.loc[(ILegal_Sum['Slot_id']  == str(Sector)) & (ILegal_Sum['Slot_Timeint'] ==str(Time_Slot))]
    a=ILegal_Sum['Ilegality_Rate'].values.tolist()
    if ILegal_Sum.empty:
        mean=0.4
    else: 
        mean=a[0]
        
    if mean==0:
        mean=0.00001
    return mean

### Main part of spatial smoothing method 

In [ ]:
def Smouth(Legal_illegal,Names):
    Time_Slots=[21600,25200,28800,32400,36000,39600,43200,46800,50400,54000,57600,61200,64800,68400,72000]
    Time_SlotsCenter=[]
    for i in range (0,len(Time_Slots)):
        Time_SlotsCenter.append((Time_Slots[i]+1800)/timedelta(days=1).total_seconds())
    Time_Slots=Time_SlotsCenter

    Scan_List2=Legal_illegal.values.tolist()
    NewData=[]
    Slots=[]
    for i in tqdm(range(0,len(Scan_List2))):
        Helper=[]
        Helper2=[]
        Rate=Scan_List2[i][6]  
        Real_Time=Scan_List2[i][5]


        Distances=[]
        for j in range (0,len(Time_Slots)):
            Distances.append(abs(Time_Slots[j]-Real_Time))
        Slots=np.column_stack((Time_Slots, Distances))
        Slots = sorted(Slots, key=lambda x: x[1])
        Slot1,Slot2,Slot3=Slots[0][0],Slots[1][0],Slots[2][0] 
        D1,D2,D3=Slots[0][1],Slots[1][1],Slots[2][1] 
        Sector=Scan_List2[i][0]    
        
        Mean=Return_Mean(Slot1,Sector,Legal_illegal,Names)
        Diff=Rate/Mean
        
        Helper=Scan_List2[i][:11]
        Helper.append(Slot1)
        Helper.append(Rate)
        Helper.append(0)
        NewData.append(Helper)       
        
        Mean=Return_Mean(Slot2,Sector,Legal_illegal,Names)
        Applied_Mean=Mean*Diff
        if Applied_Mean>1:
            Applied_Mean=1 
        
        Helper=Scan_List2[i][:11]
        Helper.append(Slot2)

        X1=((-D2)/0.14583) 
        X2=np.exp(X1)

        X3=(1-X2)*Applied_Mean
        r=(Rate*X2)+X3
    
        Helper.append(r)
        Helper.append(D2)
        NewData.append(Helper)
        
        Mean=Return_Mean(Slot3,Sector,Legal_illegal,Names)
        Applied_Mean=Mean*Diff
        if Applied_Mean>1:
            Applied_Mean=1
        
        Helper=Scan_List2[i][:11]
        Helper.append(Slot3)
        X1=((-D3)/0.14583)
        X2=np.exp(X1)
  
        X3=(1-X2)*Applied_Mean
        r=(Rate*X2)+X3
        
        Helper.append(r)
        Helper.append(D3)
        NewData.append(Helper)

    Col=['Slot_id','Key','Date_Sin','Slot_Timeint','Covid','Time_Int','Ilegality_Rate','Holidays','Capacity','Week_Day_Sin','Month_Sin','Real_Time','Real_Rate','Time_Distance']
    Legal_illegal = pd.DataFrame (NewData, columns = Col)
    
    Legal_illegal=Legal_illegal.drop(['Slot_Timeint'], axis=1)
    Legal_illegal=Legal_illegal.drop(['Time_Int'], axis=1)
    Legal_illegal=Legal_illegal.drop(['Ilegality_Rate'], axis=1)
    a=Legal_illegal['Time_Distance']
    Legal_illegal=Legal_illegal.drop(['Time_Distance'], axis=1)
    Legal_illegal.insert(8, "Time_Distance", a, True)
    return Legal_illegal

### Get weather info for the new data samples

In [ ]:
def Get_Weather(Legal_illegal):
    Time=Legal_illegal['Real_Time']*timedelta(days=1).total_seconds()
    Time=Time/3600
    Time=Time.astype(int)
    Time=Time.values.tolist()
    NewT=[]
    
    for i in range (0,len(Time)):
        Str=str(Time[i])
        if Time[i]>=10:
            NewT.append(Str)
        else:
            NewT.append('0'+Str)

    Time=pd.DataFrame(NewT,columns=["Hour"])
    Time= Time["Hour"].map(str)+ ':00'

    T_List=Legal_illegal.values.tolist()
    Date=[]
    for i in range (0,len(Legal_illegal)):
        D,H=T_List[i][1].split(' ')
        Date.append(D)

    Legal_illegal=Legal_illegal.drop(['Key'], axis=1)
    Legal_illegal.insert(1, "Key", Date, True)   
    Key_Weather=Legal_illegal['Key'].map(str) + ' ' + Time
    Legal_illegal=Legal_illegal.drop(['Key'], axis=1)  
    Legal_illegal.insert(1, "Key", Key_Weather, True)   
    Legal_illegal=pd.merge(Legal_illegal, Final_Weather_Data, on='Key')   
    return Legal_illegal

### Deduplication using mean

In [ ]:
def GetMeans(Scans_Data2):
    Scans_Data2['Key1'] = Scans_Data2['Slot_id'].map(str)+ '+' +Scans_Data2['Key'].map(str)+ '+' +Scans_Data2['Date_Sin'].map(str) + '+' + Scans_Data2['Covid'].map(str)+ '+' + Scans_Data2['Holidays'].map(str)+ '+'+Scans_Data2['Capacity'].map(str)+ '+'+Scans_Data2['Week_Day_Sin'].map(str)+ '+'+Scans_Data2['Month_Sin'].map(str)+ '+'+Scans_Data2['Real_Time'].map(str)+ '+'+Scans_Data2['temp'].map(str)+ '+'+Scans_Data2['humidity'].map(str)
    Scans_Data2=Scans_Data2['Key1'],Scans_Data2['Time_Distance'],Scans_Data2['Real_Rate']
    Headers=['Key1','Time_Distance','Real_Rate']
    Scans_Data2 = pd.concat(Scans_Data2, axis=1, keys=Headers)
    
    Time_Int = Scans_Data2.groupby(["Key1"]).Time_Distance.mean().reset_index()
    ILegal_Sum = Scans_Data2.groupby(["Key1"]).Real_Rate.mean().reset_index()
    Legal_Sum=ILegal_Sum['Key1'],Time_Int['Time_Distance'],ILegal_Sum['Real_Rate']
    Headers=['Key1','Time_Distance','Real_Rate']
    Legal_illegal = pd.concat(Legal_Sum, axis=1, keys=Headers)
    
    
    Scan_List2=Legal_illegal.values.tolist()
    Slot_id=[]
    Key=[]
    Date_Sin=[]
    Covid=[]
    Holidays=[]
    Capacity=[]
    Week_Day_Sin=[]
    Month_Sin=[]
    Real_Time=[]
    temp=[]
    humidity=[]
    
    for i in tqdm(range(0,len(Scan_List2))):
        Slot_id_V,Key_V,Date_Sin_V,Covid_V,Holidays_V,Capacity_V,Week_Day_Sin_V,Month_Sin_V,Real_Time_V,temp_V,humidity_V=Scan_List2[i][0].split('+')
        Slot_id.append(Slot_id_V)
        Key.append(Key_V)
        Date_Sin.append(Date_Sin_V)
        Covid.append(Covid_V)
        Holidays.append(Holidays_V)
        Capacity.append(Capacity_V)
        Week_Day_Sin.append(Week_Day_Sin_V)
        Month_Sin.append(Month_Sin_V)
        Real_Time.append(Real_Time_V)
        temp.append(temp_V)
        humidity.append(humidity_V)
    Legal_illegal.insert(1, "Slot_id", Slot_id, True)
    Legal_illegal.insert(2, "Key", Key, True)
    Legal_illegal.insert(3, "Date_Sin", Date_Sin, True)
    Legal_illegal.insert(4, "Covid",  Covid, True)
    Legal_illegal.insert(5, "Holidays", Holidays, True)
    Legal_illegal.insert(6, "Capacity", Capacity, True)
    Legal_illegal.insert(7, "Week_Day_Sin", Week_Day_Sin, True)
    Legal_illegal.insert(8, "Month_Sin", Month_Sin, True)
    Legal_illegal.insert(9, "Real_Time", Real_Time, True)
    Legal_illegal.insert(10, "temp", temp, True)
    Legal_illegal.insert(11, "humidity", humidity, True)
    Legal_illegal=Legal_illegal.drop(['Key1'], axis=1)
    
    Columns2=['Slot_id','Key', 'Date_Sin', 'Covid', 'Holidays', 'Capacity', 'Week_Day_Sin',
    'Month_Sin', 'Time_Distance', 'Real_Time', 'Real_Rate', 'temp','humidity']
    Legal_illegal=Legal_illegal[Columns2]
    return Legal_illegal

### Replacing sector value with Poi distance based vector

In [ ]:
def Get_Slot_Distances(Legal_illegal):
    Legal_illegal['Slot_id'] = Legal_illegal['Slot_id'].astype(float)
    Legal_illegal['Slot_id'] = Legal_illegal['Slot_id'].astype(int)
    Legal_illegal=pd.merge(Legal_illegal, Distance_Data, on='Slot_id')
    return Legal_illegal

## Save Final Train / Test Set 

In [ ]:
def Apply_Smoothing(Legal_illegal,Names):
    Legal_illegal=Smouth(Legal_illegal,Names)
    Legal_illegal=Get_Weather(Legal_illegal)
    Legal_illegal=GetMeans(Legal_illegal)
    Legal_illegal=Get_Slot_Distances(Legal_illegal)
    return Legal_illegal

In [ ]:
def Prepare_Test(Test): 
    Test['Time_Distance']=0
    Test=Test.drop(['Time_Int'], axis=1)
    a=Test['Slot_Timeint']
    b=Test['Ilegality_Rate']
    Test=Test.drop(['Slot_Timeint'], axis=1)
    Test=Test.drop(['Ilegality_Rate'], axis=1)
    Test.insert(9, "Real_Time", a, True)
    Test.insert(10, "Real_Rate", b, True)
    Test=pd.merge(Test, Final_Weather_Data, on='Key')
    Test=pd.merge(Test, Distance_Data, on='Slot_id')
    return Test

In [ ]:
TestDF=Prepare_Test(test_data) #No smoothing
TrainDF=Apply_Smoothing(train_data,Names) #Smoothing
TrainDF=TrainDF.drop(['Time_Distance'], axis=1)
TestDF=TestDF.drop(['Time_Distance'], axis=1)
TrainDF.to_csv(Project_Path+ '/Data/Ts_Train_TimeSeries.csv')
TestDF.to_csv(Project_Path+ '/Data/Ts_Test_TimeSeries.csv')

## Save Standar Scaller

In [ ]:
TrainDF=TrainDF.drop(['Slot_id'], axis=1)
TrainDF=TrainDF.drop(['Key'], axis=1)
TestDF=TestDF.drop(['Slot_id'], axis=1)
TestDF=TestDF.drop(['Key'], axis=1)
train_data=TrainDF.drop(['Real_Rate'], axis=1)
test_data=TestDF.drop(['Real_Rate'], axis=1)

Standar_Scaller = StandardScaler()
train_data=Standar_Scaller.fit_transform(train_data)
with open(Project_Path + '/Standar_Scaller.pkl', 'wb') as f:
    pickle.dump(Standar_Scaller, f,  protocol=2)